In [31]:
import os
import sys
import pandas as pd
from sklearn import datasets
import statsmodels.api as sm
from stargazer.stargazer import Stargazer
from IPython.display import display, HTML
from sklearn.datasets import load_iris
from pystout import pystout
import matplotlib.pyplot as plt

import numpy as np

In [32]:
# Global Settings
pd.set_option("display.float_format", lambda x: "%.4f" % x)

In [33]:
cwd = os.getcwd()
# Find and import config file
config_path = os.getcwd()

sys.path.append(config_path)
import config

database = config.database
central_banks = config.central_banks
training_data = os.path.join(database, "Training Data")
fed_docs = config.fed_docs
ecb_docs = config.ecb_docs
boe_docs = config.boe_docs

sentiment = pd.date_range(start="1/1/1990", end="1/1/2024", freq="D")
sentiment = pd.DataFrame(sentiment, columns=["date"])

In [34]:
sentiment = pd.date_range(start="1/1/1990", end="12/31/2024", freq="D")
sentiment = pd.DataFrame(sentiment, columns=["date"])

In [35]:
# Import inflation probability data that is stored as a dta file
inflation_prob_us = pd.read_stata(
    "/Users/kylenabors/Documents/Database/Other Data/Inflation Probabilities/USwestimates.dta"
)

inflation_prob_us = inflation_prob_us.rename(columns={"date_ym": "date"})

inflation_prob_eu = pd.read_stata(
    "/Users/kylenabors/Documents/Database/Other Data/Inflation Probabilities/EZwestimates.dta"
)

inflation_prob_eu = inflation_prob_eu.rename(columns={"date_ym": "date"})

inflation_prob = inflation_prob_us.merge(
    inflation_prob_eu, on="date", how="left", suffixes=("_us", "_eu")
)

sentiment = sentiment.merge(inflation_prob, on="date", how="left")

sentiment = sentiment.ffill()

In [36]:
# # HP Filter

url_map = pd.read_csv(os.path.join(cwd, "url_map.csv"))

for i in range(len(url_map)):
    temp_dates = pd.date_range(start="1/1/1990", end="12/31/2024", freq="D")
    temp_dates = pd.DataFrame(temp_dates, columns=["date"])
    sent = pd.read_csv(url_map["finbert_url"][i])
    sent = sent[["date", "sentiment"]]

    sent["date"] = pd.to_datetime(sent["date"])
    sent = sent.groupby("date").mean().reset_index()

    sent = pd.merge(temp_dates, sent, how="left", left_on="date", right_on="date")

    # Set 'date' as the index for resampling
    sent.set_index("date", inplace=True)

    # Drop NaNs
    sent = sent.dropna(subset=["sentiment"])

    # Reset index to turn 'date' back into a column
    sent.reset_index(inplace=True)

    filter_df = sent.copy(deep=True)

    cycle, trend = sm.tsa.filters.hpfilter(
        filter_df["sentiment"], 1600 * ((8 / 4) ** 4)
    )

    filter_df["sentiment_cycle"] = cycle
    filter_df["sentiment_trend"] = trend

    filter_df = filter_df[["date", "sentiment_cycle"]]
    sent = sent.drop(columns=["sentiment"])
    filter_df = filter_df.rename(columns={"sentiment_cycle": "sentiment"})
    sent = pd.merge(sent, filter_df, on="date", how="outer")
    sent = sent.groupby("date").mean().reset_index()

    sent = sent.rename(columns={"sentiment": url_map["document"][i]})
    sentiment = pd.merge(sentiment, sent, how="outer", left_on="date", right_on="date")

In [37]:
market = pd.read_csv(f"{database}/Market Data/All Market Data.csv")
market = market.rename(columns={"PCE": "pce", "GDP": "gdp", "GDPPOT": "gdp_pot"})
market["date"] = pd.to_datetime(market["date"])

sentiment = pd.merge(sentiment, market, how="left", left_on="date", right_on="date")

In [38]:
fed_sentiment = sentiment.copy(deep=True)
fed_sentiment = fed_sentiment[["date", "fed_minutes"]]
fed_sentiment = fed_sentiment.dropna(subset=["fed_minutes"])

fed_sentiment["count"] = range(len(fed_sentiment))

# fed_sentiment["mod"] = fed_sentiment["count"] % 2
# fed_sentiment["count"] = fed_sentiment["count"] + fed_sentiment["mod"]

fed_sentiment = fed_sentiment[["date", "count"]]
sentiment = pd.merge(
    sentiment, fed_sentiment, how="left", left_on="date", right_on="date"
)
sentiment["count"] = sentiment["count"].ffill()

fed_sentiment = sentiment.copy(deep=True)
fed_sentiment = fed_sentiment[["count", "sp500_return"]]
fed_sentiment["log_returns"] = np.log(fed_sentiment["sp500_return"] + 1)
fed_sentiment = fed_sentiment[["count", "log_returns"]]
fed_sentiment = fed_sentiment.groupby("count").sum().reset_index()
fed_sentiment = fed_sentiment.groupby(fed_sentiment["count"]).mean().reset_index()

fed_temp = sentiment.copy(deep=True)
fed_temp = fed_temp[
    [
        "count",
        "fed_minutes",
        "press_conferences",
        "higher4_5y5y_us",
        "lower0_5y5y_us",
        "zc_higher4_5y_us",
        "zc_lower0_5y_us",
    ]
]

fed_temp["press_conferences"] = fed_temp["press_conferences"].ffill()
fed_temp = fed_temp.dropna(subset=["fed_minutes"])

fed_sentiment = pd.merge(
    fed_sentiment, fed_temp, how="left", left_on="count", right_on="count"
)

In [39]:
ecb_sentiment = sentiment.copy(deep=True)
ecb_sentiment = ecb_sentiment[["date", "press_conferences"]]
ecb_sentiment = ecb_sentiment.dropna(subset=["press_conferences"])

ecb_sentiment["ecb_count"] = range(len(ecb_sentiment))

# ecb_sentiment["mod"] = ecb_sentiment["ecb_count"] % 2
# ecb_sentiment["ecb_count"] = ecb_sentiment["ecb_count"] + ecb_sentiment["mod"]

ecb_sentiment = ecb_sentiment[["date", "ecb_count"]]
sentiment = pd.merge(
    sentiment, ecb_sentiment, how="left", left_on="date", right_on="date"
)
sentiment["ecb_count"] = sentiment["ecb_count"].ffill()

ecb_sentiment = sentiment.copy(deep=True)
ecb_sentiment = ecb_sentiment[["ecb_count", "stoxx"]]
ecb_sentiment["log_returns_eu"] = np.log(ecb_sentiment["stoxx"] + 1)
ecb_sentiment = ecb_sentiment[["ecb_count", "log_returns_eu"]]
ecb_sentiment = ecb_sentiment.groupby("ecb_count").sum().reset_index()
ecb_sentiment = ecb_sentiment.groupby(ecb_sentiment["ecb_count"]).mean().reset_index()

ecb_temp = sentiment.copy(deep=True)
ecb_temp = ecb_temp[
    [
        "ecb_count",
        "press_conferences",
        "fed_minutes",
        "higher4_5y5y_eu",
        "lower0_5y5y_eu",
        "zc_higher4_5y_eu",
        "zc_lower0_5y_eu",
    ]
]
ecb_temp["fed_minutes"] = ecb_temp["fed_minutes"].ffill()
ecb_temp = ecb_temp.dropna(subset=["press_conferences"])

ecb_sentiment = pd.merge(
    ecb_sentiment, ecb_temp, how="left", left_on="ecb_count", right_on="ecb_count"
)

In [40]:
# Create lagged columns
for column in sentiment.columns:
    if column != "date":  # Skip the 'date' column
        sentiment[f"{column}_1"] = sentiment[column].shift(1)


for i in range(0, 6):
    fed_sentiment[f"log_returns_{i}"] = fed_sentiment["log_returns"].shift(i)
    ecb_sentiment[f"log_returns_eu_{i}"] = ecb_sentiment["log_returns_eu"].shift(i)

for i in range(0, 2):
    fed_sentiment[f"higher4_5y5y_us_{i}"] = fed_sentiment["higher4_5y5y_us"].shift(i)
    fed_sentiment[f"lower0_5y5y_us_{i}"] = fed_sentiment["lower0_5y5y_us"].shift(i)
    fed_sentiment[f"zc_higher4_5y_us_{i}"] = fed_sentiment["zc_higher4_5y_us"].shift(i)
    fed_sentiment[f"zc_lower0_5y_us_{i}"] = fed_sentiment["zc_lower0_5y_us"].shift(i)
    ecb_sentiment[f"higher4_5y5y_eu_{i}"] = ecb_sentiment["higher4_5y5y_eu"].shift(i)
    ecb_sentiment[f"lower0_5y5y_eu_{i}"] = ecb_sentiment["lower0_5y5y_eu"].shift(i)
    ecb_sentiment[f"zc_higher4_5y_eu_{i}"] = ecb_sentiment["zc_higher4_5y_eu"].shift(i)
    ecb_sentiment[f"zc_lower0_5y_eu_{i}"] = ecb_sentiment["zc_lower0_5y_eu"].shift(i)


# The count runs from day of announcment till day before next announcment so we shift the values by 1 so fed_minutes is now t where as before it was t-1

fed_sentiment["fed_minutes"] = fed_sentiment["fed_minutes"].shift(-1)
fed_sentiment["fed_minutes_1"] = fed_sentiment["fed_minutes"].shift(1)
ecb_sentiment["press_conferences"] = ecb_sentiment["press_conferences"].shift(-1)
ecb_sentiment["press_conferences_1"] = ecb_sentiment["press_conferences"].shift(1)

ecb_sentiment["fed_minutes"] = ecb_sentiment["fed_minutes"].shift(-1)
ecb_sentiment["fed_minutes_1"] = ecb_sentiment["fed_minutes"].shift(1)
fed_sentiment["press_conferences"] = fed_sentiment["press_conferences"].shift(-1)
fed_sentiment["press_conferences_1"] = fed_sentiment["press_conferences"].shift(1)

for i in range(1, 5):
    fed_sentiment[f"fed_minutes_forward_{i}"] = fed_sentiment["fed_minutes"].shift(-i)
    ecb_sentiment[f"press_conferences_forward_{i}"] = ecb_sentiment[
        "press_conferences"
    ].shift(-i)

In [41]:
fed_sentiment = fed_sentiment.dropna()
ecb_sentiment = ecb_sentiment.dropna()

fed_sentiment = fed_sentiment[fed_sentiment["log_returns_5"] != 0]
ecb_sentiment = ecb_sentiment[ecb_sentiment["log_returns_eu_5"] != 0]

In [42]:
sentiment = sentiment[
    (sentiment["date"] >= "2000-01-01") & (sentiment["date"] <= "2023-06-30")
]

In [43]:
x0 = [
    "fed_minutes_1",
    "log_returns_0",
    "log_returns_1",
    "log_returns_2",
    "log_returns_3",
    "log_returns_4",
    "log_returns_5",
    # "log_returns_eu_6",
    # "log_returns_eu_7",
]

x1 = [
    "press_conferences_1",
    "log_returns_eu_0",
    "log_returns_eu_1",
    "log_returns_eu_2",
    "log_returns_eu_3",
    "log_returns_eu_4",
    "log_returns_eu_5",
]

# x2 = [
#     "minute_sentiment_1",
#     "log_returns_eu_0",
#     "log_returns_eu_2",
#     "log_returns_eu_4",
#     "log_returns_eu_6",
# ]

# x3 = [
#     "pc_sentiment_1",
#     "log_returnse_us_0",
#     "log_returnse_us_2",
#     "log_returnse_us_4",
#     "log_returnse_us_6",
# ]

yvar = "fed_minutes"
yvar1 = "press_conferences"
# yvar2 = "minute_sentiment"
# yvar3 = "pc_sentiment"

exog0 = (sm.add_constant(fed_sentiment[x0])).dropna()
exog1 = (sm.add_constant(ecb_sentiment[x1])).dropna()
# exog2 = (sm.add_constant(sentiment_mb[x2])).dropna()
# exog3 = (sm.add_constant(sentiment_mbe[x3])).dropna()

reg0 = sm.OLS(endog=fed_sentiment[yvar].loc[exog0.index], exog=exog0).fit(
    cov_type="HC0"
)
reg1 = sm.OLS(endog=ecb_sentiment[yvar1].loc[exog1.index], exog=exog1).fit(
    cov_type="HC0"
)
# reg2 = sm.OLS(endog=sentiment_mb[yvar2].loc[exog2.index], exog=exog2).fit(
#     cov_type="HC0"
# )
# reg3 = sm.OLS(endog=sentiment_mbe[yvar3].loc[exog3.index], exog=exog3).fit(
#     cov_type="HC0"
# )

stargazer = Stargazer([reg0, reg1])
stargazer.title(
    "Regressing Fed Sentiment on SP500 Returns and Regressing ECB Sentiment on STOXX 600 Returns"
)
stargazer.custom_columns(["Fed(t)", "ECB(t)"])
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)
stargazer.covariate_order(
    [
        "log_returns_0",
        "log_returns_1",
        "log_returns_2",
        "log_returns_3",
        "log_returns_4",
        "log_returns_5",
        "log_returns_eu_0",
        "log_returns_eu_1",
        "log_returns_eu_2",
        "log_returns_eu_3",
        "log_returns_eu_4",
        "log_returns_eu_5",
        "fed_minutes_1",
        "press_conferences_1",
    ]
)
stargazer.rename_covariates(
    {
        "log_returns_0": "SP00 Returns(t)",
        "log_returns_1": "SP00 Returns(t-1)",
        "log_returns_2": "SP00 Returns(t-2)",
        "log_returns_3": "SP00 Returns(t-3)",
        "log_returns_4": "SP00 Returns(t-4)",
        "log_returns_5": "SP00 Returns(t-5)",
        "fed_minutes_1": "Fed(t-1)",
        "log_returns_eu_0": "STOXX600 Returns(t)",
        "log_returns_eu_1": "STOXX600 Returns(t-1)",
        "log_returns_eu_2": "STOXX600 Returns(t-2)",
        "log_returns_eu_3": "STOXX600 Returns(t-3)",
        "log_returns_eu_4": "STOXX600 Returns(t-4)",
        "log_returns_eu_5": "STOXX600 Returns(t-5)",
        "press_conferences_1": "ECB(t-1)",
    },
)
stargazer.add_custom_notes(
    [
        "The unit for t is the time between a given meeting and the previous meeting for the given Central Bank."
    ]
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)

\begin{table}[H] \centering
  \caption{Regressing Fed Sentiment on SP500 Returns and Regressing ECB Sentiment on STOXX 600 Returns}
\begin{tabular}{@{\extracolsep{5pt}}lcc}
\\hline
\hline \
\ & \multicolumn{1}{c}{Fed(t)} & \multicolumn{1}{c}{ECB(t)}  \\
\hline \
 SP00 Returns(t) & 0.035$^{}$ & \\
& (0.064) & \\
 SP00 Returns(t-1) & 0.159$^{**}$ & \\
& (0.078) & \\
 SP00 Returns(t-2) & 0.366$^{***}$ & \\
& (0.063) & \\
 SP00 Returns(t-3) & 0.156$^{**}$ & \\
& (0.077) & \\
 SP00 Returns(t-4) & 0.109$^{}$ & \\
& (0.074) & \\
 SP00 Returns(t-5) & 0.025$^{}$ & \\
& (0.072) & \\
 STOXX600 Returns(t) & & -0.137$^{*}$ \\
& & (0.072) \\
 STOXX600 Returns(t-1) & & 0.082$^{}$ \\
& & (0.129) \\
 STOXX600 Returns(t-2) & & 0.177$^{**}$ \\
& & (0.088) \\
 STOXX600 Returns(t-3) & & 0.168$^{*}$ \\
& & (0.101) \\
 STOXX600 Returns(t-4) & & 0.075$^{}$ \\
& & (0.091) \\
 STOXX600 Returns(t-5) & & -0.053$^{}$ \\
& & (0.085) \\
 Fed(t-1) & 0.341$^{***}$ & \\
& (0.075) & \\
 ECB(t-1) & & 0.513$^{***}$ \\
& &

In [44]:
# Fed SR LR ECB SR LR

x0 = [
    "higher4_5y5y_us",
    "higher4_5y5y_us_1",
    "lower0_5y5y_us",
    "lower0_5y5y_us_1",
    "fed_minutes_1",
    "log_returns_0",
    "log_returns_1",
    "log_returns_2",
    "log_returns_3",
    "log_returns_4",
    "log_returns_5",
]

x1 = [
    "zc_higher4_5y_us",
    "zc_higher4_5y_us_1",
    "zc_lower0_5y_us",
    "zc_lower0_5y_us_1",
    "fed_minutes_1",
    "log_returns_0",
    "log_returns_1",
    "log_returns_2",
    "log_returns_3",
    "log_returns_4",
    "log_returns_5",
]

x2 = [
    "higher4_5y5y_eu",
    "higher4_5y5y_eu_1",
    "lower0_5y5y_eu",
    "lower0_5y5y_eu_1",
    "press_conferences_1",
    "log_returns_eu_0",
    "log_returns_eu_1",
    "log_returns_eu_2",
    "log_returns_eu_3",
    "log_returns_eu_4",
    "log_returns_eu_5",
]

x3 = [
    "zc_higher4_5y_eu",
    "zc_higher4_5y_eu_1",
    "zc_lower0_5y_eu",
    "zc_lower0_5y_eu_1",
    "press_conferences_1",
    "log_returns_eu_0",
    "log_returns_eu_1",
    "log_returns_eu_2",
    "log_returns_eu_3",
    "log_returns_eu_4",
    "log_returns_eu_5",
]


yvar = "fed_minutes"
yvar1 = "fed_minutes"
yvar2 = "press_conferences"
yvar3 = "press_conferences"

exog0 = (sm.add_constant(fed_sentiment[x0])).dropna()
exog1 = (sm.add_constant(fed_sentiment[x1])).dropna()
exog2 = (sm.add_constant(ecb_sentiment[x2])).dropna()
exog3 = (sm.add_constant(ecb_sentiment[x3])).dropna()

reg0 = sm.OLS(endog=fed_sentiment[yvar].loc[exog0.index], exog=exog0).fit(
    cov_type="HC0"
)
reg1 = sm.OLS(endog=fed_sentiment[yvar1].loc[exog1.index], exog=exog1).fit(
    cov_type="HC0"
)
reg2 = sm.OLS(endog=ecb_sentiment[yvar2].loc[exog2.index], exog=exog2).fit(
    cov_type="HC0"
)
reg3 = sm.OLS(endog=ecb_sentiment[yvar3].loc[exog3.index], exog=exog3).fit(
    cov_type="HC0"
)

stargazer = Stargazer([reg0, reg1, reg2, reg3])
stargazer.title(
    "Regressing Fed Sentiment on SP500 Returns and Regressing ECB Sentiment on STOXX 600 Returns"
)
stargazer.custom_columns(["Fed(t)", "Fed(t)", "ECB(t)", "ECB(t)"])
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)
stargazer.covariate_order(
    [
        "higher4_5y5y_us",
        "higher4_5y5y_us_1",
        "lower0_5y5y_us",
        "lower0_5y5y_us_1",
        "zc_higher4_5y_us",
        "zc_higher4_5y_us_1",
        "zc_lower0_5y_us",
        "zc_lower0_5y_us_1",
        "higher4_5y5y_eu",
        "higher4_5y5y_eu_1",
        "lower0_5y5y_eu",
        "lower0_5y5y_eu_1",
        "zc_higher4_5y_eu",
        "zc_higher4_5y_eu_1",
        "zc_lower0_5y_eu",
        "zc_lower0_5y_eu_1",
        "log_returns_0",
        "log_returns_1",
        "log_returns_2",
        "log_returns_3",
        "log_returns_4",
        "log_returns_5",
        "log_returns_eu_0",
        "log_returns_eu_1",
        "log_returns_eu_2",
        "log_returns_eu_3",
        "log_returns_eu_4",
        "log_returns_eu_5",
        "fed_minutes_1",
        "press_conferences_1",
    ]
)
stargazer.rename_covariates(
    {
        "higher4_5y5y_us": "Inflation(t)",
        "higher4_5y5y_us_1": "Inflation(t-1)",
        "lower0_5y5y_us": "Deflation(t)",
        "lower0_5y5y_us_1": "Deflation(t-1)",
        "zc_higher4_5y_us": "Inflation(t)",
        "zc_higher4_5y_us_1": "Inflation(t-1)",
        "zc_lower0_5y_us": "Deflation(t)",
        "zc_lower0_5y_us_1": "Deflation(t-1)",
        "higher4_5y5y_eu": "Inflation(t)",
        "higher4_5y5y_eu_1": "Inflation(t-1)",
        "lower0_5y5y_eu": "Deflation(t)",
        "lower0_5y5y_eu_1": "Deflation(t-1)",
        "zc_higher4_5y_eu": "Inflation(t)",
        "zc_higher4_5y_eu_1": "Inflation(t-1)",
        "zc_lower0_5y_eu": "Deflation(t)",
        "zc_lower0_5y_eu_1": "Deflation(t)",
        "log_returns_0": "SP00 Returns(t)",
        "log_returns_1": "SP00 Returns(t-1)",
        "log_returns_2": "SP00 Returns(t-2)",
        "log_returns_3": "SP00 Returns(t-3)",
        "log_returns_4": "SP00 Returns(t-4)",
        "log_returns_5": "SP00 Returns(t-5)",
        "fed_minutes_1": "Fed(t-1)",
        "log_returns_eu_0": "STOXX600 Returns(t)",
        "log_returns_eu_1": "STOXX600 Returns(t-1)",
        "log_returns_eu_2": "STOXX600 Returns(t-2)",
        "log_returns_eu_3": "STOXX600 Returns(t-3)",
        "log_returns_eu_4": "STOXX600 Returns(t-4)",
        "log_returns_eu_5": "STOXX600 Returns(t-5)",
        "press_conferences_1": "ECB(t-1)",
    },
)
stargazer.add_custom_notes(
    [
        "The unit for t is the time between a given meeting and the previous meeting for the given Central Bank."
    ]
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)

\begin{table}[H] \centering
  \caption{Regressing Fed Sentiment on SP500 Returns and Regressing ECB Sentiment on STOXX 600 Returns}
\begin{tabular}{@{\extracolsep{5pt}}lcccc}
\\hline
\hline \
\ & \multicolumn{1}{c}{Fed(t)} & \multicolumn{1}{c}{Fed(t)} & \multicolumn{1}{c}{ECB(t)} & \multicolumn{1}{c}{ECB(t)}  \\
\hline \
 Inflation(t) & -0.015$^{}$ & & & \\
& (0.503) & & & \\
 Inflation(t-1) & 0.481$^{}$ & & & \\
& (0.443) & & & \\
 Deflation(t) & -0.505$^{}$ & & & \\
& (0.446) & & & \\
 Deflation(t-1) & -0.291$^{}$ & & & \\
& (0.393) & & & \\
 Inflation(t) & & 0.128$^{}$ & & \\
& & (0.186) & & \\
 Inflation(t-1) & & -0.032$^{}$ & & \\
& & (0.163) & & \\
 Deflation(t) & & -2.052$^{***}$ & & \\
& & (0.492) & & \\
 Deflation(t-1) & & 1.470$^{***}$ & & \\
& & (0.500) & & \\
 Inflation(t) & & & 0.722$^{}$ & \\
& & & (1.023) & \\
 Inflation(t-1) & & & -1.130$^{}$ & \\
& & & (1.081) & \\
 Deflation(t) & & & -0.569$^{*}$ & \\
& & & (0.299) & \\
 Deflation(t-1) & & & 0.132$^{}$ & \\
& & & (0.3

In [45]:
# Table 3 Pt. 1

x = [
    "press_conferences",
    "press_conferences_1",
    "fed_minutes_1",
]

yvar0 = "fed_minutes"
yvar1 = "fed_minutes_forward_1"
yvar2 = "fed_minutes_forward_2"
yvar3 = "fed_minutes_forward_3"
yvar4 = "fed_minutes_forward_4"


# Prepare the exogenous variables for each regression
exog0 = sm.add_constant(fed_sentiment[x]).dropna()
exog1 = sm.add_constant(fed_sentiment[x]).dropna()
exog2 = sm.add_constant(fed_sentiment[x]).dropna()
exog3 = sm.add_constant(fed_sentiment[x]).dropna()
exog4 = sm.add_constant(fed_sentiment[x]).dropna()


# Fit the regression models
reg0 = sm.OLS(endog=fed_sentiment[yvar0].loc[exog0.index], exog=exog0).fit(
    cov_type="HC0"
)
reg1 = sm.OLS(endog=fed_sentiment[yvar1].loc[exog1.index], exog=exog1).fit(
    cov_type="HC0"
)
reg2 = sm.OLS(endog=fed_sentiment[yvar2].loc[exog2.index], exog=exog2).fit(
    cov_type="HC0"
)
reg3 = sm.OLS(endog=fed_sentiment[yvar3].loc[exog3.index], exog=exog3).fit(
    cov_type="HC0"
)
reg4 = sm.OLS(endog=fed_sentiment[yvar4].loc[exog4.index], exog=exog4).fit(
    cov_type="HC0"
)


# Create the Stargazer object
stargazer = Stargazer([reg0, reg1, reg2, reg3, reg4])

# Customize the Stargazer output
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)

stargazer.title("Fed Minute Sentiment Regressed on Financial Variables")
stargazer.custom_columns(
    [
        "Fed Sent. (t)",
        "Fed Sent. (t+1)",
        "Fed Sent. (t+2)",
        "Fed Sent. (t+3)",
        "Fed Sent. (t+4)",
    ]
)

stargazer.covariate_order(
    [
        "press_conferences",
        "press_conferences_1",
        "fed_minutes_1",
    ]
)

stargazer.rename_covariates(
    {
        "press_conferences": "ECB Sent (t)",
        "press_conferences_1": "ECB Sent (t-1)",
        "fed_minutes_1": "Fed Sent (t-1)",
    }
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)

\begin{table}[H] \centering
  \caption{Fed Minute Sentiment Regressed on Financial Variables}
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\hline
\hline \
\ & \multicolumn{1}{c}{Fed Sent. (t)} & \multicolumn{1}{c}{Fed Sent. (t+1)} & \multicolumn{1}{c}{Fed Sent. (t+2)} & \multicolumn{1}{c}{Fed Sent. (t+3)} & \multicolumn{1}{c}{Fed Sent. (t+4)}  \\
\hline \
 ECB Sent (t) & 0.159$^{**}$ & 0.208$^{***}$ & 0.168$^{**}$ & 0.018$^{}$ & -0.020$^{}$ \\
& (0.074) & (0.072) & (0.077) & (0.075) & (0.083) \\
 ECB Sent (t-1) & 0.058$^{}$ & -0.045$^{}$ & -0.134$^{}$ & -0.107$^{}$ & -0.064$^{}$ \\
& (0.076) & (0.080) & (0.082) & (0.085) & (0.090) \\
 Fed Sent (t-1) & 0.404$^{***}$ & 0.312$^{***}$ & 0.175$^{*}$ & 0.213$^{*}$ & 0.060$^{}$ \\
& (0.083) & (0.081) & (0.090) & (0.109) & (0.101) \\
\hline \
 Observations & 101 & 101 & 101 & 101 & 101 \\
 $R^2$ & 0.289 & 0.190 & 0.073 & 0.047 & 0.011 \\
 Adjusted $R^2$ & 0.267 & 0.165 & 0.044 & 0.017 & -0.020 \\
 Residual Std. Error & 0.070 (df=97) & 0.074 (d

In [46]:
# Table 3 Pt. 2

x = [
    "fed_minutes",
    "fed_minutes_1",
    "press_conferences_1",
]

yvar0 = "press_conferences"
yvar1 = "press_conferences_forward_1"
yvar2 = "press_conferences_forward_2"
yvar3 = "press_conferences_forward_3"
yvar4 = "press_conferences_forward_4"


# Prepare the exogenous variables for each regression
exog0 = sm.add_constant(ecb_sentiment[x]).dropna()
exog1 = sm.add_constant(ecb_sentiment[x]).dropna()
exog2 = sm.add_constant(ecb_sentiment[x]).dropna()
exog3 = sm.add_constant(ecb_sentiment[x]).dropna()
exog4 = sm.add_constant(ecb_sentiment[x]).dropna()


# Fit the regression models
reg0 = sm.OLS(endog=ecb_sentiment[yvar0].loc[exog0.index], exog=exog0).fit(
    cov_type="HC0"
)
reg1 = sm.OLS(endog=ecb_sentiment[yvar1].loc[exog1.index], exog=exog1).fit(
    cov_type="HC0"
)
reg2 = sm.OLS(endog=ecb_sentiment[yvar2].loc[exog2.index], exog=exog2).fit(
    cov_type="HC0"
)
reg3 = sm.OLS(endog=ecb_sentiment[yvar3].loc[exog3.index], exog=exog3).fit(
    cov_type="HC0"
)
reg4 = sm.OLS(endog=ecb_sentiment[yvar4].loc[exog4.index], exog=exog4).fit(
    cov_type="HC0"
)


# Create the Stargazer object
stargazer = Stargazer([reg0, reg1, reg2, reg3, reg4])

# Customize the Stargazer output
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)

stargazer.title("Fed Minute Sentiment Regressed on Financial Variables")
stargazer.custom_columns(
    [
        "ECB Sent. (t)",
        "ECB Sent. (t+1)",
        "ECB Sent. (t+2)",
        "ECB Sent. (t+3)",
        "ECB Sent. (t+4)",
    ]
)

stargazer.covariate_order(
    [
        "fed_minutes",
        "fed_minutes_1",
        "press_conferences_1",
    ]
)

stargazer.rename_covariates(
    {
        "fed_minutes": "Fed Sent (t)",
        "fed_minutes_1": "Fed Sent (t-1)",
        "press_conferences_1": "ECB Sent (t-1)",
    }
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)

\begin{table}[H] \centering
  \caption{Fed Minute Sentiment Regressed on Financial Variables}
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\hline
\hline \
\ & \multicolumn{1}{c}{ECB Sent. (t)} & \multicolumn{1}{c}{ECB Sent. (t+1)} & \multicolumn{1}{c}{ECB Sent. (t+2)} & \multicolumn{1}{c}{ECB Sent. (t+3)} & \multicolumn{1}{c}{ECB Sent. (t+4)}  \\
\hline \
 Fed Sent (t) & 0.160$^{}$ & 0.374$^{**}$ & 0.191$^{}$ & 0.133$^{}$ & 0.161$^{}$ \\
& (0.158) & (0.168) & (0.151) & (0.126) & (0.139) \\
 Fed Sent (t-1) & 0.135$^{}$ & -0.044$^{}$ & 0.070$^{}$ & 0.116$^{}$ & 0.054$^{}$ \\
& (0.141) & (0.160) & (0.159) & (0.125) & (0.135) \\
 ECB Sent (t-1) & 0.464$^{***}$ & 0.203$^{**}$ & 0.035$^{}$ & -0.039$^{}$ & -0.024$^{}$ \\
& (0.089) & (0.085) & (0.101) & (0.091) & (0.085) \\
\hline \
 Observations & 111 & 111 & 111 & 111 & 111 \\
 $R^2$ & 0.316 & 0.140 & 0.039 & 0.027 & 0.023 \\
 Adjusted $R^2$ & 0.297 & 0.116 & 0.012 & -0.000 & -0.005 \\
 Residual Std. Error & 0.094 (df=107) & 0.105 (df=107) &